# 用于预训练BERT的数据集

In [1]:
import os
import random
import torch
from d2l import torch as d2l

在WikiText-2数据集中，每行代表一个段落，其中在任意标点符号及其前面的词元之间插入空格。保留至少有两句话的段落。为了简单起见，我们仅使用句号作为分隔符来拆分句子。我们将更复杂的句子拆分技术的讨论留在本节末尾的练习中。


In [2]:
#@save
d2l.DATA_HUB['wikitext-2'] = (
    'https://s3.amazonaws.com/research.metamind.io/wikitext/'
    'wikitext-2-v1.zip', '3c914d17d80b1459be871a5039ac23e752a53cbe')

#@save
def _read_wiki(data_dir):
    '''
    1. 构建文件路径
    data_dir：数据目录路径
    wiki.train.tokens：WikiText格式的训练文件
    文件格式：纯文本，每行是一个段落，句子用 .（空格+句点+空格）分隔
    '''

    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    '''
    2. 读取文件
    读取所有行：每行是一个段落字符串
    lines：列表，每个元素是一个字符串（包含换行符）
    '''
    with open(file_name, 'r') as f:
        lines = f.readlines()
    # 大写字母转换为小写字母
    '''
    3. 处理段落
    步骤1：line.strip()：移除行首尾的空白字符和换行符
    步骤2：.lower()：大写转小写：统一文本格式，减少词汇表大小
    例："Hello World . This is BERT." → "hello world . this is bert."
    步骤3：.split(' . ')：按句子分隔符分割，使用' . '（空格+句点+空格）作为分隔符，结果得到一个句子列表
    例："hello world . this is bert ." → ['hello world', 'this is bert', '']
    步骤4：if len(...)>=2，筛选有效段落 ：只保留至少包含2个句子的段落，目的：为NSP任务准备（需要句对）
    '''
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    '''
    4. 随机打乱
    作用：随机打乱段落顺序
    目的：确保训练数据分布均匀，避免模型记住顺序
    '''
    random.shuffle(paragraphs)
    # 返回结果：paragraphs列表，每个元素是句子列表
    return paragraphs

## 为预训练任务定义辅助函数
### 生成下一句预测任务的数据

| 代码行                         | 功能       | 关键技术 |
| --------------------------- | -------- | ---- |
| `random.random() < 0.5`     | 50%正例/负例 | 随机采样 |
| `random.choice(paragraphs)` | 选择随机段落   | 双重随机 |
| `random.choice(...)`        | 选择随机句子   | 负例构造 |
| `is_next = True/False`      | 生成标签     | 监督信号 |


In [3]:
'''
sentence：当前句子A（字符串）
next_sentence：句子A的真实下一句（正例）
paragraphs：所有段落的列表（三重嵌套结构）
'''
#@save
def _get_next_sentence(sentence, next_sentence, paragraphs):
    # 50%概率生成正例
    if random.random() < 0.5: # 生成[0,1)之间的随机数，以50%概率保持真实下一句
        is_next = True # 标签为正（是下一句
    else:
        # 50%概率生成负例
        # paragraphs是三重列表的嵌套
        next_sentence = random.choice(random.choice(paragraphs)) # 随机选择一个段落（二级列表），再在该段落中随机选择一个句子
        is_next = False # 标签为负（不是下一句）
    return sentence, next_sentence, is_next # next_sentence是真实的

下面的函数通过调用`_get_next_sentence`函数从输入`paragraph`生成用于下一句预测的训练样本。这里`paragraph`是句子列表，其中每个句子都是词元列表。自变量`max_len`指定预训练期间的BERT输入序列的最大长度。


**完整数据流动示例**

**输入段落**

```Python
paragraph = [
    'the cat sat on the mat',   # 句子0
    'it was raining outside',    # 句子1 (真实下一句)
    'the dog barked loudly'      # 句子2
]
paragraphs = [paragraph, ...]  # 所有段落
max_len = 10
```
**处理过程**

**处理句对 (句子0, 句子1)**

```Python
# 情况1: 50%概率生成正例
tokens_a = ['the', 'cat', 'sat', 'on', 'the', 'mat']
tokens_b = ['it', 'was', 'raining', 'outside']
is_next = True

# 检查长度: 6 + 4 + 3 = 13 > 10 → 跳过! (continue)

# 情况2: 50%概率生成负例
tokens_a = ['the', 'cat', 'sat', 'on', 'the', 'mat']
tokens_b = ['transformer', 'is', 'powerful']  # 来自随机段落
is_next = False

# 检查长度: 6 + 3 + 3 = 12 > 10 → 跳过! (continue)
```
**处理句对 (句子1, 句子2)**
```Python
tokens_a = ['it', 'was', 'raining', 'outside']
tokens_b = ['the', 'dog', 'barked', 'loudly']

# 检查长度: 4 + 4 + 3 = 11 > 10 → 跳过! (continue)
结果：这个段落不生成任何样本，因为所有句对都太长。
```

| 代码行                                             | 功能       | 关键技术          |
| ----------------------------------------------- | -------- | ------------- |
| `range(len(paragraph) - 1)`             | 遍历句对     | 相邻索引          |
|  `_get_next_sentence(...)`              | 生成正/负例   | 50%采样         |
|  `+ 3 > max_len`                        | 长度过滤     | 特殊词元计数        |
|  `get_tokens_and_segments`              | BERT格式转换 | <cls> + <sep> |
|  `append((tokens, segments, is_next))`  | 存储样本     | 三元组           |
|  `return nsp_data_from_paragraph`       | 返回结果     | 列表            |


In [4]:
'''
以50%概率生成正例（真实下一句）
以50%概率生成负例（随机段落中的随机句子）
检查长度约束并转换为BERT输入格式
'''
#@save
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    # 1. 初始化结果列表：存储当前段落生成的所有NSP训练样本
    nsp_data_from_paragraph = []
    '''
    2. 遍历段落中的句子
        遍历索引0到len(paragraph)-1
        原因：需要取paragraph[i]和paragraph[i+1]组成句对
    '''
    for i in range(len(paragraph) - 1):
        '''
        3. 生成正例/负例
        paragraph[i]：当前句子A（字符串）
        paragraph[i+1]：句子A的真实下一句B（字符串）
        paragraphs：所有段落的嵌套列表（用于负例采样）
        返回：
            tokens_a：句子A的词元列表
            tokens_b：句子B的词元列表（可能是真实下一句，也可能是随机句子）
            is_next：布尔值，True表示是真实下一句，False表示随机句子
        '''
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        '''
        4. 长度检查与过滤
        +3的含义：
            '<cls>'：1个特殊词元（序列开头）
            '<sep>'：2个特殊词元（句子A结尾、句子B结尾）
        max_len：最大序列长度限制（如512）
        continue：如果超长，跳过该句对，不加入训练集
        '''
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        '''
        5. 转换为BERT输入格式
            tokens：合并后的词元列表（格式：<cls>+tokens_a+<sep>+tokens_b+<sep>）
            segments：段标记列表（0表示句子A，1表示句子B）
        '''
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        '''
        6. 存储样本
        tokens：输入词元序列
        segments：片段标记
        is_next：标签（True/False）
        '''
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    # 7. 返回所有样本：返回值：列表，每个元素是一个(tokens,segments,is_next)元组
    return nsp_data_from_paragraph

### 生成遮蔽语言模型任务的数据


**完整示例演示**

**输入**
```Python
tokens = ['<cls>', 'the', 'cat', 'sat', 'on', 'the', 'mat', '<sep>']
candidate_pred_positions = [1, 2, 3, 4, 5, 6]  # 排除<cls>和<sep>
num_mlm_preds = 3  # 掩蔽15% ~ 3个词元
vocab.idx_to_token = ['<cls>', '<sep>', '<mask>', 'the', 'cat', 'sat', 'on', 'mat', ...]
```
**执行过程（假设的随机数）**

| 位置               | 原始词    | random() | 替换策略    | masked\_token | 是否记录 |
| ---------------- | ------ | -------- | ------- | ------------- | ---- |
|  2(cat)  | `<0.8` | `<mask>` | ✅       | 是             |      |
|  4(on)   | `0.85` | 随机词      | `'dog'` | ✅             |      |
|  5(the)  | `0.95` | 保持原词     | `'the'` | ✅             |      |

**结果：**
```Python
mlm_input_tokens = ['<cls>', 'the', '<mask>', 'sat', 'dog', 'the', 'mat', '<sep>']

pred_positions_and_labels = [
    (2, 'cat'),   # 位置2的原始词是'cat'
    (4, 'on'),    # 位置4的原始词是'on'
    (5, 'the')    # 位置5的原始词是'the'
]
```
**损失计算示例**
```Python
# mlm_Y_hat: 预测logits (batch, num_preds, vocab_size)
# pred_positions_and_labels: 真实标签

# 提取真实词索引
mlm_Y = [label for _, label in pred_positions_and_labels]  # ['cat', 'on', 'the']
mlm_Y_idx = [vocab[token] for token in mlm_Y]  # 转换为索引

# 计算损失
loss = nn.CrossEntropyLoss()
mlm_loss = loss(mlm_Y_hat.reshape(-1, vocab_size), mlm_Y_idx)
```
**关键设计原理**

**1. 为什么需要三种替换策略？**

仅用<mask>会导致训练和推理不一致（推理时没有<mask>）。80-10-10策略平衡了：
- **主要信号**（80%）：学习预测被掩蔽词
- **平滑过渡**（10%保持）：减少训练和推理差距
- **鲁棒性**（10%随机）：防止过度依赖<mask>

2. 为什么打乱位置顺序？

```Python
random.shuffle(candidate_pred_positions)
```
- 避免模式学习：如果总是按顺序掩蔽前15%位置，模型可能学到位置模式而非语义
- 随机性：更贴近真实世界的噪声

3. 为什么限制掩蔽数量？

```Python
if len(pred_positions_and_labels) >= num_mlm_preds: break
```
- 控制比例：通常为15%（如序列长度20 → 掩蔽3个）
- 计算效率：过多的掩蔽会增加训练时间和难度

| 代码片段                                        | 功能           | 设计目的      |
| ------------------------------------------- | ------------ | --------- |
| `[token for token in tokens]`       | 创建副本         | 保护原始数据    |
|  `random.shuffle(...)`              | 打乱位置         | 增加随机性     |
|  `if random.random() < 0.8`         | 80%替换为<mask> | 主要训练信号    |
|  `elif random.random() < 0.5`       | 10%保持原词      | 减少训练-推理差距 |
|  `else: random.choice(...)`         | 10%随机词       | 提升鲁棒性     |
|  `pred_positions_and_labels.append` | 记录标签         | 监督学习需要    |


```txt
[  expression  for  variable  in  iterable  ]
   ╲________╱   ╲_______╱    ╲_________╱
       │           │              │
       │           │              └─ 要遍历的原始列表（tokens）
       │           └─ 临时变量（每次循环的当前元素）
       └─ 最终存入新列表的值
```

```txt
随机选择一个词元位置
        ↓
   ┌────┴────┐
   │  random() │
   └────┬────┘
        ↓
   ┌────┴────┐
   │  < 0.8?  │
   └──┬───┬───┘
      │   │
  ┌───┘   └────┐
  │            │
  ▼            ▼
80%概率      20%概率
替换为mask    进入else
              │
        ┌─────┴─────┐
        │ random() < 0.5? │
        └──┬───┬────┘
           │   │
       ┌───┘   └───┐
       │           │
       ▼           ▼
    10%概率      10%概率
   保持原词     随机替换
```

| 步骤       | 操作                                                 | 结果                                                            |
| -------- | -------------------------------------------------- | ------------------------------------------------------------- |
| 初始化      | `mlm_input_tokens = [token for token in tokens]`   | `['<cls>', '我', '爱', '深度', '学习', '<sep>']`                    |
| 随机选择位置3  | `random.shuffle()`后取第一个                            | `mlm_pred_position = 3`                                       |
| 80%概率判断  | `random.random() < 0.8` 为True                      | `masked_token = '<mask>'`                                     |
| **执行替换** | `mlm_input_tokens[3] = '<mask>'`                   | `['<cls>', '我', '爱', '<mask>', '学习', '<sep>']`                |
| **记录标签** | `pred_positions_and_labels.append((3, tokens[3]))` | `[(3, '深度')]`                                                 |
| 返回       | `return ...`                                       | `(['<cls>', '我', '爱', '<mask>', '学习', '<sep>'], [(3, '深度')])` |


In [5]:
'''
tokens:BERT输入序列的词元列表（如['<cls>','我','爱','深','度','学','习','<sep>']）
candidate_pred_positions:可能被遮蔽的词元位置索引列表（特殊词元如<cls>、<sep>会被排除）
num_mlm_preds:需要预测的词元数量（通常是序列长度的15%）
vocab:词表对象，用于随机选择替换词元
'''
#@save
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds,
                        vocab):
    # 为遮蔽语言模型的输入创建新的词元副本，其中输入可能包含替换的“<mask>”或随机词元
    '''
    1. 创建输入词元副本
    目的：创建可修改的副本，不破坏原始序列
    深拷贝：列表推导式确保修改副本不影响原始tokens
    '''
    mlm_input_tokens = [token for token in tokens]
    '''
    2. 初始化预测位置和标签列表
    pred_positions_and_labels：存储预测位置和对应的真实词元
    用途：存储被替换位置及其原始词元的元组
    格式：[(位置1,原始词1),(位置2,原始词2),...]
    '''
    pred_positions_and_labels = []
    # 打乱后用于在遮蔽语言模型任务中获取15%的随机词元进行预测
    '''
    3. 打乱候选预测位置
    原因：确保随机性，模型无法预测哪些位置会被掩蔽
    candidate_pred_positions：所有可被掩蔽的位置索引列表（通常排除<cls>和<sep>）
    '''
    random.shuffle(candidate_pred_positions)
    '''
    4. 遍历并选择预测位置
    终止条件：达到预设的掩蔽数量（通常为序列长度的15%）
    提前退出：避免掩蔽过多词元
    '''
    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        '''
        5. 80-10-10替换策略（核心）
        这是BERT MLM任务的关键设计，对每个候选位置：
        (1) 80%的时间：将词替换为“<mask>”词元
            主要任务：模型必须预测原始词元
            训练目标：学习双向上下文理解
        '''
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            '''
            (2) 10%的时间：保持词不变
                目的：减少训练和推理差异（推理时没有<mask>）
                信号：模型看到原始词元，但仍需预测它（有点矛盾，但有效）
            '''
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            else:
                '''
                (3) 10%概率替换为随机词
                    目的：强制模型不完全依赖<mask>，必须理解上下文
                    噪声引入：随机词提供干扰，防止模型"偷懒"
                '''
                masked_token = random.choice(vocab.idx_to_token)
        '''
        6. 执行替换并记录 
        替换操作：将词元放入副本的指定位置
        记录标签：保存原始位置和原始词元，用于后续损失计算
        '''
        mlm_input_tokens[mlm_pred_position] = masked_token
        '''
        作用：将被遮蔽的位置和该位置的原始词元组成元组，存入列表
        为什么用tokens而不是mlm_input_tokens？
            tokens[mlm_pred_position]：原始词元（如'学习'），作为预测目标（label）
            mlm_input_tokens[mlm_pred_position]：已被替换为<mask>或随机词，不再是正确答案
        数据结构：[(pos1,label1),(pos2,label2),...]，确保位置与标签一一对应
        '''
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    '''
    返回值1：mlm_input_tokens-替换后的词元序列（用于模型输入）
    返回值2：pred_positions_and_labels-被遮蔽位置和原始词元的配对列表（用于计算损失）
    '''
    return mlm_input_tokens, pred_positions_and_labels

通过调用前述的`_replace_mlm_tokens`函数，以下函数将BERT输入序列（`tokens`）作为输入，并返回输入词元的索引（在 `subsec_mlm`中描述的可能的词元替换之后）、发生预测的词元索引以及这些预测的标签索引。


**完整数据流示例**

**输入：**
```Python
tokens = ['<cls>', '我', '爱', '深度', '学习', '<sep>']
vocab = {'<cls>': 101, '我': 2769, '爱': 4263, '深度': 6207, '学习': 2603, '<sep>': 102, ...}
```
**处理过程：**
```Python
candidate_pred_positions = [1, 2, 3, 4]  # 排除0和5
num_mlm_preds = max(1, round(6 * 0.15)) = 1

# 调用 _replace_mlm_tokens (假设随机选中位置3，80%概率替换为mask)
mlm_input_tokens = ['<cls>', '我', '爱', '<mask>', '学习', '<sep>']
pred_positions_and_labels = [(3, '深度')]

# 排序（单个元素无需变化）
pred_positions_and_labels = [(3, '深度')]

# 分离
pred_positions = [3]
mlm_pred_labels = ['深度']

# 转为ID
return [101, 2769, 4263, 103, 2603, 102], [3], [6207]
```
**与模型的交互**

返回的三个值将被用于：
```python
vocab[mlm_input_tokens] → 作为模型的输入序列
pred_positions → 告诉模型在哪些位置做预测
vocab[mlm_pred_labels] → 作为监督信号（ground truth）
```
在模型前向传播中：

```Python
# encoded_X 是BERT编码后的表示
masked_X = encoded_X[batch_idx, pred_positions]  # 只取出需要预测的位置
mlm_Y_hat = self.mlp(masked_X)  # 预测每个位置的词元

# 计算损失
loss = cross_entropy(mlm_Y_hat, mlm_pred_labels)  # 与真实标签对比
```

In [6]:
#@save
def _get_mlm_data_from_tokens(tokens, vocab):
    # 1. 初始化候选位置列表：用于存储所有可以被遮蔽的词元位置索引
    candidate_pred_positions = []
    # tokens是一个字符串列表
    '''
    2. 遍历词元序列，排除特殊词元
    关键逻辑：特殊词元（<cls>分类标记、<sep>分隔标记）不参与预测
    原因：这些词元是BERT输入的结构标记，不是语义内容
    结果：candidate_pred_positions=[1,2,3,4]（假设tokens长度为5）
    '''
    for i, token in enumerate(tokens):
        # 在遮蔽语言模型任务中不会预测特殊词元
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)
    '''
    3. 计算需要预测的词元数量
    遮蔽语言模型任务中预测15%的随机词元
    规则：至少预测1个词元（即使序列很短）
    比例：15%的词元会被选中
    示例：若len(tokens)=20，则num_mlm_preds=3
    max(1,...)的作用：防止序列过短时计算结果为0
    '''
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    '''
    4. 执行遮蔽替换
    调用下层函数：传入原始词元、候选位置、预测数量和词表
    返回结果：
    mlm_input_tokens：已遮蔽的词元序列（如['<cls>','我','<mask>','NLP','<sep>']）
    pred_positions_and_labels：被遮蔽位置和原始词元的配对列表，如[(2,'爱')]
    '''
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(
        tokens, candidate_pred_positions, num_mlm_preds, vocab)
    '''
    5. 按位置排序（关键步骤）
    作用：确保预测位置按升序排列
    为什么需要排序？
        _replace_mlm_tokens中random.shuffle()打乱了选择顺序
        模型需要固定顺序来处理预测位置
        后续分离位置/标签时保证一一对应关系不混乱
    '''
    pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
    # 分离位置和标签：将元组列表拆分为两个独立列表
    pred_positions = [v[0] for v in pred_positions_and_labels] # 提取位置
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels] # 提取原始词元
    '''
    返回最终数据
    返回值1：vocab[mlm_input_tokens]-输入词元的ID序列
        通过词表将词元转换为整数ID（如[101,2769,103,1920,102]）
    返回值2：pred_positions-被遮蔽位置的索引列表
    返回值3：vocab[mlm_pred_labels]-预测目标的ID列表
    '''
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

## 将文本转换为预训练数据集

现在我们几乎准备好为BERT预训练定制一个`Dataset`类。在此之前，我们仍然需要定义辅助函数`_pad_bert_inputs`来将特殊的“&lt;mask&gt;”词元附加到输入。它的参数`examples`包含来自两个预训练任务的辅助函数`_get_nsp_data_from_paragraph`和`_get_mlm_data_from_tokens`的输出。


| 返回值                  | 形状 (batch\_size, ...)             | 说明            |
| -------------------- | --------------------------------- | ------------- |
| `all_token_ids`      | `(batch_size, max_len)`           | 填充后的词元ID序列    |
| `all_segments`       | `(batch_size, max_len)`           | 段落标记（区分句子A/B） |
| `valid_lens`         | `(batch_size,)`                   | 每个样本的真实长度     |
| `all_pred_positions` | `(batch_size, max_num_mlm_preds)` | 需要预测的位置索引     |
| `all_mlm_weights`    | `(batch_size, max_num_mlm_preds)` | **屏蔽填充位置的权重** |
| `all_mlm_labels`     | `(batch_size, max_num_mlm_preds)` | 被遮蔽词元的标签ID    |
| `nsp_labels`         | `(batch_size,)`                   | 下一句预测的标签      |


**假设处理2个样本：**

```Python
examples = [
    # 样本1：长度5，预测1个位置
    ([101, 2769, 4263, 6207, 102], [2], [4263], [0,0,0,0,0], 1),
    # 样本2：长度3，预测1个位置  
    ([101, 2603, 102], [1], [2603], [0,0,0], 0)
]
max_len = 5
max_num_mlm_preds = 1
```
**填充后结果：**
```Python
all_token_ids = [
    [101, 2769, 4263, 6207, 102],  # 样本1（无需填充）
    [101, 2603, 102, 0, 0]          # 样本2（填充2个pad）
]

all_pred_positions = [
    [2],    # 样本1的预测位置
    [1]     # 样本2的预测位置
]

all_mlm_weights = [
    [1.0],  # 样本1的真实预测
    [1.0]   # 样本2的真实预测
]

# 如果max_num_mlm_preds=2，则：
all_pred_positions = [
    [2, 0],     # 第二个位置是填充
    [1, 0]      # 第二个位置是填充
]
all_mlm_weights = [
    [1.0, 0.0],  # 第二个权重为0
    [1.0, 0.0]   # 第二个权重为0
]
```

In [7]:
'''
_pad_bert_inputs将多个样本（每个样本长度不同）填充到固定的max_len，并返回7个列表，分别对应：
    token序列、段落标记、有效长度
    预测位置、预测权重、预测标签
    NSP标签
'''
#@save
def _pad_bert_inputs(examples, max_len, vocab):
    '''
    作用：计算每个样本最多需要预测多少词元（15%的max_len）
    示例：若max_len=128，则max_num_mlm_preds=19
    '''
    max_num_mlm_preds = round(max_len * 0.15)
    all_token_ids, all_segments, valid_lens,  = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    
    for (token_ids, pred_positions, mlm_pred_label_ids, segments,
         is_next) in examples:
        # 1.  填充token序列：在序列末尾添加<pad>词元，直到长度达到max_len
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (
            max_len - len(token_ids)), dtype=torch.long)) # 差值
        # 2. 填充段落标记：段落标记也填充为0（与pad保持一致），这里的0表示"属于段落A"，填充部分也属于A
        all_segments.append(torch.tensor(segments + [0] * (
            max_len - len(segments)), dtype=torch.long))
        # 3. 记录有效长度（不含pad）：记录原始序列的真实长度（不包括填充部分）
        # 示例：len(token_ids)=5，有效长度就是5
        # 用途：在Transformer的Attention计算中，用valid_len屏蔽pad位置
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        # 4. 填充预测位置：预测位置列表也填充到固定长度max_num_mlm_preds
        # 填充值：用0填充（0是有效位置，但后面用权重屏蔽）
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (
            max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # 填充词元的预测将通过乘以0权重在损失中过滤掉
        '''
        5. 设计预测权重
        作用：区分真实预测位置和填充位置
        机制：
            真实预测位置→权重为1.0（参与损失计算）
            填充位置→权重为0.0（在损失中被屏蔽）
        实现：在计算损失时，loss*weight，填充位置的loss被置0
        '''
        all_mlm_weights.append(
            torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (
                max_num_mlm_preds - len(pred_positions)),
                dtype=torch.float32))
        # 6. 填充预测标签：标签也填充为0（配合权重为0，不会影响损失）
        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (
            max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
        # 7. 收集NSP标签：收集"是否为下一句"的二分类标签（0或1）
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    return (all_token_ids, all_segments, valid_lens, all_pred_positions,
            all_mlm_weights, all_mlm_labels, nsp_labels)

将用于生成两个预训练任务的训练样本的辅助函数和用于填充输入的辅助函数放在一起，我们定义以下`_WikiTextDataset`类为用于预训练BERT的WikiText-2数据集。通过实现`__getitem__ `函数，我们可以任意访问WikiText-2语料库的一对句子生成的预训练样本（遮蔽语言模型和下一句预测）样本。

最初的BERT模型使用词表大小为30000的WordPiece嵌入 :`Wu.Schuster.Chen.ea.2016`。WordPiece的词元化方法是对 `subsec_Byte_Pair_Encoding`中原有的字节对编码算法稍作修改。为简单起见，我们使用`d2l.tokenize`函数进行词元化。出现次数少于5次的不频繁词元将被过滤掉。


**完整数据流示例**
```Python
# 输入
paragraphs = [
    "I love NLP. It's fun.", 
    "Deep learning rocks! Machine learning too."
]

# Step 1: 词元化
paragraphs = [
    [['i', 'love', 'nlp', '.'], ['it', "'s", 'fun', '.']],
    [['deep', 'learning', 'rocks', '!'], ['machine', 'learning', 'too', '.']]
]

# Step 2: 构建词表
sentences = ['i', 'love', ..., 'too', '.']
vocab = Vocab(..., reserved_tokens=['<pad>', '<mask>', '<cls>', '<sep>'])

# Step 3: 生成NSP样本 (简化)
examples = [
    ([<cls>, i, love, nlp, <sep>, deep, learning, <sep>], [0,0,0,0,0,1,1,1], 1),  # 正例
    ([<cls>, i, love, nlp, <sep>, machine, <sep>, <pad>], [0,0,0,0,0,1,1,0], 0)   # 负例
]

# Step 4: 生成MLM遮蔽 (假设遮蔽位置2和5)
examples = [
    ([<cls>, i, <mask>, nlp, <sep>, deep, <mask>, <sep>], [2,5], [love,learning], [0,0,0,0,0,1,1,1], 1),
    # ...其他样本
]

# Step 5: 填充到max_len (假设max_len=10)
self.all_token_ids = [
    [101, 2769, 103, 6207, 102, 2362, 103, 102, 0, 0],  # 后面两个0是pad
    # ...其他样本
]
self.valid_lens = [8, 7, ...]  # 不含pad的真实长度
```

| 序号 | 返回值                            | 形状示例 (max\_len=8, max\_pred=2)         | 说明              |
| -- | ------------------------------ | -------------------------------------- | --------------- |
| 0  | `self.all_token_ids[idx]`      | `[101, 2769, 103, 6207, 102, 0, 0, 0]` | 输入词元ID（填充后）     |
| 1  | `self.all_segments[idx]`       | `[0, 0, 0, 0, 0, 0, 0, 0]`             | 段落标记（0=A句,1=B句） |
| 2  | `self.valid_lens[idx]`         | `5` (标量)                               | 有效长度（不含pad）     |
| 3  | `self.all_pred_positions[idx]` | `[2, 0]`                               | MLM预测位置索引       |
| 4  | `self.all_mlm_weights[idx]`    | `[1.0, 0.0]`                           | 预测权重（屏蔽填充）      |
| 5  | `self.all_mlm_labels[idx]`     | `[4263, 0]`                            | MLM预测标签         |
| 6  | `self.nsp_labels[idx]`         | `1` (标量)                               | NSP任务标签（0/1）    |


In [8]:
#@save
class _WikiTextDataset(torch.utils.data.Dataset):
    '''
    paragraphs:原始段落列表，每个元素是字符串（如["I love NLP. It's fun.","Deep learning rocks!"]）
    max_len:最大序列长度（如128、512）
    '''
    def __init__(self, paragraphs, max_len):
        # 输入paragraphs[i]是代表段落的句子字符串列表；
        # 而输出paragraphs[i]是代表段落的句子列表，其中每个句子都是词元列表
        # 1.  词元化（Tokenization）：将每个段落字符串拆分为句子列表，再将每个句子拆分为词元列表
        paragraphs = [d2l.tokenize(
            paragraph, token='word') for paragraph in paragraphs]
        '''
        2. 构建词汇表
        双重列表推导式：将所有段落的所有句子展平为一个句子列表
        min_freq=5：只保留出现≥5次的词（过滤低频词）
        reserved_tokens：强制添加的特殊词元：
            <pad>：填充标记
            <mask>：遮蔽标记
            <cls>：分类标记（句首）
            <sep>：分隔标记（句间）
        '''
        sentences = [sentence for paragraph in paragraphs
                     for sentence in paragraph]
        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>', '<mask>', '<cls>', '<sep>'])
        '''
        3. 生成NSP（下一句预测）数据
        调用函数：_get_nsp_data_from_paragraph为每个段落生成NSP样本
        样本格式：每个样本是 (tokens,segments,is_next)
            tokens：词元ID列表（如[<cls>,句子A,<sep>,句子B,<sep>]）
            segments：段落标记（0/1，区分句子A/B）
            is_next：二分类标签（1=是下一句，0=不是）
        作用：构建BERT的第一个预训练任务数据
        '''
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(
                paragraph, paragraphs, self.vocab, max_len))
        '''
        4. 生成MLM（遮蔽语言模型）数据
        操作：对每个NSP样本，生成MLM遮蔽数据
        _get_mlm_data_from_tokens 返回：(mlm_input_tokens,pred_positions,mlm_pred_labels)
        元组拼接：将MLM数据与NSP数据合并
        最终格式：(tokens,pred_positions,mlm_labels,segments,is_next)
        '''
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab)
                      + (segments, is_next))
                     for tokens, segments, is_next in examples]
        # 填充输入
        '''
        5. 填充并存储数据
        调用 _pad_bert_inputs：将所有样本填充到max_len长度
        解压赋值：将返回的7个列表分别赋值给类的属性
        最终属性：
            self.all_token_ids：输入词元ID序列（填充后）
            self.all_segments：段落标记（填充后）
            self.valid_lens：有效长度（不含填充）
            self.all_pred_positions：预测位置（填充后）
            self.all_mlm_weights：预测权重（1.0/0.0屏蔽填充）
            self.all_mlm_labels：预测标签（填充后）
            self.nsp_labels：NSP二分类标签
        '''
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(
            examples, max_len, self.vocab) # 元组解包赋值
    '''
    功能：通过索引idx获取单个训练样本
    调用方式：dataset[i]等价于dataset.__getitem__(i)
    返回：包含7个张量的元组，对应一个BERT训练样本
    '''
    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx], self.all_pred_positions[idx],
                self.all_mlm_weights[idx], self.all_mlm_labels[idx],
                self.nsp_labels[idx])
    '''
    作用：返回数据集中有多少个训练样本
    原理：self.all_token_ids是一个列表，每个元素对应一个样本，所以它的长度就是样本总数
    '''
    def __len__(self):
        return len(self.all_token_ids)

通过使用`_read_wiki`函数和`_WikiTextDataset`类，我们定义了下面的`load_data_wiki`来下载并生成WikiText-2数据集，并从中生成预训练样本。


In [9]:
'''
作用：加载WikiText-2语料库，返回可迭代的数据加载器和词表
batch_size：每个批次的样本数量（如256、512）
max_len：序列最大长度（如128、512）
'''

#@save
def load_data_wiki(batch_size, max_len):
    """加载WikiText-2数据集"""
    '''
    1. 设置数据加载工作进程
    作用：获取CPU核心数，用于并行数据加载
    示例：8核CPU→num_workers=4（留出核心给主进程）
    目的：加速数据预处理，避免GPU等待数据
    '''
    num_workers = 0
    '''
    2. 下载并解压数据集
    行为：自动从网络下载WikiText-2数据集并解压
    WikiText-2是什么：包含维基百科文章的中等规模语料库（训练集约36MB）
    返回：解压后的文件夹路径
    '''
    data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
    '''
    3. 读取原始文本
    调用_read_wiki：读取所有文本文件，按段落组织
    返回格式：paragraphs[i]是第i个段落的原始字符串
    '''
    paragraphs = _read_wiki(data_dir)
    '''
    4.  创建数据集实例
    调用__init__：执行我们之前解析的完整数据处理流水线
    耗时操作：词表构建、NSP/MLM样本生成、填充处理（可能耗时数分钟）
    结果：train_set包含所有预处理后的数据（数百万样本）
    '''
    train_set = _WikiTextDataset(paragraphs, max_len)
    '''
    5. 创建数据迭代器
    作用：将数据集包装为可迭代对象，支持批量加载
    关键参数：
    shuffle=True：每个epoch打乱样本顺序，防止模型学到顺序偏见
    num_workers=num_workers：多进程并行加载数据
    返回：train_iter是PyTorch的DataLoader对象
    '''
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                        shuffle=True, num_workers=num_workers)
    '''
    6. 返回结果
    返回值1：train_iter-数据加载器，训练时循环迭代
    返回值2：train_set.vocab-词表，用于词元↔ID转换
    '''
    return train_iter, train_set.vocab

将批量大小设置为512，将BERT输入序列的最大长度设置为64，我们打印出小批量的BERT预训练样本的形状。注意，在每个BERT输入序列中，为遮蔽语言模型任务预测$10$（$64 \times 0.15$）个位置。


| 变量名                | 形状示例                    | 含义                           |
| ------------------ | ----------------------- | ---------------------------- |
| `tokens_X`         | `torch.Size([512, 64])` | **输入词元ID**（512样本 × 64长度）     |
| `segments_X`       | `torch.Size([512, 64])` | **段落标记**（0/1表示句子A/B）         |
| `valid_lens_x`     | `torch.Size([512])`     | **有效长度**（每个样本的真实长度）          |
| `pred_positions_X` | `torch.Size([512, 9])`  | **预测位置**（9 = round(64×0.15)） |
| `mlm_weights_X`    | `torch.Size([512, 9])`  | **预测权重**（1.0/0.0屏蔽填充）        |
| `mlm_Y`            | `torch.Size([512, 9])`  | **MLM标签**（被遮蔽词的真实ID）         |
| `nsp_y`            | `torch.Size([512])`     | **NSP标签**（0/1表示是否为下一句）       |


In [10]:
'''
batch_size=512：每个批次包含512个训练样本
max_len=64：每个样本的最大序列长度（包括<cls>、<sep>和填充）
train_iter：PyTorch的DataLoader对象，可迭代
vocab：词表，用于后续解码
'''
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)
# 批数据解包与迭代
for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X,
     mlm_Y, nsp_y) in train_iter:
    # 批数据解包与迭代
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape,
          pred_positions_X.shape, mlm_weights_X.shape, mlm_Y.shape,
          nsp_y.shape)
    break # 只打印第一个批次就退出

torch.Size([512, 64]) torch.Size([512, 64]) torch.Size([512]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512])


最后，我们来看一下词量。即使在过滤掉不频繁的词元之后，它仍然比PTB数据集的大两倍以上。


In [11]:
len(vocab)

20256